In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder.appName("Ecommerce").getOrCreate()

customer_path = '/Volumes/workspace/e-commerce_dataset/ecommerce_dataset/customers_dataset.csv'
product_path = '/Volumes/workspace/e-commerce_dataset/ecommerce_dataset/products_dataset.csv'
transcation_path = '/Volumes/workspace/e-commerce_dataset/ecommerce_dataset/transactions_dataset.csv'

customer_df = spark.read.csv(customer_path, header=True, inferSchema=True)
product_df = spark.read.csv(product_path, header=True, inferSchema=True)
transcation_df = spark.read.csv(transcation_path, header=True, inferSchema=True)

print("product Dataset")
product_df.show(5)
print("Trasncation Dataset")
transcation_df.show(5)
print("Customer Dataset")
customer_df.show(5)

product Dataset
+----------+------------+--------+------+
|product_id|product_name|category| price|
+----------+------------+--------+------+
|       500|       Dream|Clothing|479.56|
|       501|        Wall|Clothing|296.92|
|       502|     Service|    Home|803.57|
|       503|         All|Clothing|968.25|
|       504|   Executive|   Books|977.36|
+----------+------------+--------+------+
only showing top 5 rows
Trasncation Dataset
+--------------------+-----------+----------+------+------------+-------------------+-------+
|      transaction_id|customer_id|product_id|amount|payment_type|   transaction_time| status|
+--------------------+-----------+----------+------+------------+-------------------+-------+
|cf38974e-3bf8-474...|       1272|       545|119.67|          CC|2025-02-13 07:50:23|SUCCESS|
|8e5c41e5-0d65-438...|       1055|       608| 672.2|      wallet|2025-06-16 01:13:46|SUCCESS|
|84198714-05c0-43e...|       1164|       697|536.45|  NetBanking|2025-08-07 18:37:12| FAILED

In [0]:
transcation_dataenrich = transcation_df.withColumn("Transcation_date", to_date(col("transaction_time"))) \
                            .withColumn("amount", col("amount").cast(DoubleType()))
                                
enriched_data = (
    transcation_dataenrich
    .na.drop(subset=["transaction_id", "customer_id", "product_id"])
    .dropDuplicates(["transaction_id"])
                                      
                                        .withColumn("Payment_Types", when(col("payment_type").isin("CC","CreditCard","CREDIT_CARD","cc","credit"), "CREDIT CARD"). 
                                                    when(col("payment_type").isin('DC',"debit","DEBIT_CARD","DebitCard"), "DEBIT CARD").
                                                    when(col("payment_type").isin("NetBanking","NET_BANKING"), "NET BANKING").
                                                    when(col("payment_type").isin("WALLET","wallet"), "WALLET").
                                                    when(col("payment_type").isin("UPI","Upi"), "UPI").
                                                    otherwise("CASH ON DELIVERY"))).select("transaction_id","customer_id","product_id","amount","transaction_time","Transcation_date","Payment_Types","status")

transcation_dataenrich.show(5)
print("Enriched data")
enriched_data.show(5)
                    

+--------------------+-----------+----------+------+------------+-------------------+-------+----------------+
|      transaction_id|customer_id|product_id|amount|payment_type|   transaction_time| status|Transcation_date|
+--------------------+-----------+----------+------+------------+-------------------+-------+----------------+
|cf38974e-3bf8-474...|       1272|       545|119.67|          CC|2025-02-13 07:50:23|SUCCESS|      2025-02-13|
|8e5c41e5-0d65-438...|       1055|       608| 672.2|      wallet|2025-06-16 01:13:46|SUCCESS|      2025-06-16|
|84198714-05c0-43e...|       1164|       697|536.45|  NetBanking|2025-08-07 18:37:12| FAILED|      2025-08-07|
|db41697f-a3fa-446...|       1457|       655|568.77|  DEBIT_CARD|2025-06-29 07:29:32|SUCCESS|      2025-06-29|
|5e9c3e4a-b2ce-458...|       1081|       616|665.65|  CreditCard|2025-08-14 02:44:39|SUCCESS|      2025-08-14|
+--------------------+-----------+----------+------+------------+-------------------+-------+----------------+
o

In [0]:
# joining the data
fact_transactions = enriched_data.join(broadcast(customer_df),  on = "customer_id", how = "inner").join(broadcast(product_df), on = "product_id", how = "inner").selectExpr("*")
            

fact_transactions.show(5)
# DBTITLE 1 Write data to delta lake
fact_transactions.write.format("delta").mode("overwrite").save("/Volumes/workspace/e-commerce_dataset/ecommerce_dataset/fact_transactions")

+----------+-----------+--------------------+------+-------------------+----------------+-------------+-------+------------------+--------------------+------+------------+-----------+-------+
|product_id|customer_id|      transaction_id|amount|   transaction_time|Transcation_date|Payment_Types| status|     customer_name|               email|region|product_name|   category|  price|
+----------+-----------+--------------------+------+-------------------+----------------+-------------+-------+------------------+--------------------+------+------------+-----------+-------+
|       508|       1046|dbfcbcb9-389d-49e...|160.04|2025-04-25 06:54:17|      2025-04-25|       WALLET|SUCCESS|    Brandy Ramirez|rogeralexander@ex...|  East|        Look|Electronics|1509.44|
|       681|       1089|1e4e0c6e-7bec-45e...|130.79|2025-05-15 23:51:18|      2025-05-15|  CREDIT CARD|SUCCESS|       Shane White|andrewalexander@e...| South|         Gas|       Home|1791.78|
|       636|       1310|8d463994-57ef-46

In [0]:
%sql

select * from delta.`/Volumes/workspace/e-commerce_dataset/ecommerce_dataset/fact_transactions/`
limit 10

product_id,customer_id,transaction_id,amount,transaction_time,Transcation_date,Payment_Types,status,customer_name,email,region,product_name,category,price
508,1046,dbfcbcb9-389d-49e8-9725-7f65a727b015,160.04,2025-04-25T06:54:17.000Z,2025-04-25,WALLET,SUCCESS,Brandy Ramirez,rogeralexander@example.org,East,Look,Electronics,1509.44
681,1089,1e4e0c6e-7bec-45ec-bb34-c3be8b5e5c9f,130.79,2025-05-15T23:51:18.000Z,2025-05-15,CREDIT CARD,SUCCESS,Shane White,andrewalexander@example.org,South,Gas,Home,1791.78
636,1310,8d463994-57ef-46a1-b838-e3090b15699e,480.07,2025-07-18T06:40:46.000Z,2025-07-18,WALLET,SUCCESS,Courtney Porter MD,carrierandolph@example.com,West,Ability,Electronics,1130.71
510,1178,abfa8c56-951e-47c0-a0ff-07fba2ca0375,47.24,2025-06-23T15:31:59.000Z,2025-06-23,WALLET,FRAUD,Angel Burns,larrylee@example.org,South,Record,Clothing,316.97
516,1425,cf7403ee-02ac-4109-9ed7-0abbc04449f4,83.98,2025-03-28T00:29:48.000Z,2025-03-28,CREDIT CARD,SUCCESS,Tamara Hardin,markyoung@example.net,North,Spring,Books,282.68
522,1054,1da5b8fd-21df-4076-b134-2e00ec4c2c2d,490.94,2025-07-24T19:46:22.000Z,2025-07-24,CREDIT CARD,SUCCESS,Chelsea Brown,scooley@example.com,South,Enter,Clothing,846.96
630,1236,b28a4475-6ca5-42c5-a26e-61408f7a0c4d,929.59,2025-01-18T19:10:55.000Z,2025-01-18,CREDIT CARD,FAILED,Andrew Rojas,danielconnor@example.org,South,Election,Electronics,1592.58
646,1231,38c445ec-c495-4199-bef9-942804f8bde6,242.63,2025-09-04T15:46:12.000Z,2025-09-04,NET BANKING,SUCCESS,Adam Young,jeffreysimon@example.org,East,Improve,Books,1516.61
563,1078,c1b3e587-e60a-491c-8697-d025c42c1674,212.97,2025-06-17T13:42:42.000Z,2025-06-17,DEBIT CARD,SUCCESS,Joseph Anderson,oaguilar@example.net,South,Car,Home,71.43
528,1176,be00a86f-a8ce-4965-83a9-c639761cbaaa,663.02,2025-07-17T07:55:19.000Z,2025-07-17,CREDIT CARD,FAILED,Franklin Kim,paynejennifer@example.net,South,Message,Clothing,1164.36


In [0]:
%sql

create or replace temporary view transcation_date_fact 
as
select * from delta.`/Volumes/workspace/e-commerce_dataset/ecommerce_dataset/fact_transactions/`


In [0]:
%sql
select distinct status from transcation_date_fact

status
FRAUD
SUCCESS
FAILED


In [0]:
%sql

-- Count of suspicious/fraud transactions (status = 'FRAUD') per region.

select region, count(status) as suspicious_transcation 
from transcation_date_fact 
where status = 'FRAUD'
group by 1

region,suspicious_transcation
West,132
East,119
North,146
South,133


In [0]:
%sql

-- Total revenue (SUM(amount)) per product category and region.

select category, region, round(sum(amount),2) as total_revenue
from transcation_date_fact
group by 1,2

category,region,total_revenue
Clothing,West,342071.23
Clothing,South,306576.93
Electronics,East,210871.55
Books,South,212992.77
Home,South,220389.91
Books,East,190569.91
Sports,North,297671.38
Sports,South,255278.75
Sports,West,304794.41
Home,North,229059.78


In [0]:
%sql

-- Top 5 customers by total spend.

select customer_name, round(sum(amount)) as total_spend
from transcation_date_fact
group by 1
order by 2 desc
limit 5

customer_name,total_spend
Jason Smith,23105.0
Katherine Johnson,20151.0
Annette Johnson,20134.0
Madison Jackson MD,18598.0
Ryan Hernandez,18326.0


In [0]:
%sql

-- Percentage of transactions by payment_type (normalized → CREDIT_CARD, DEBIT_CARD, UPI, NET_BANKING, WALLET):

select Payment_Types, round(count(*)*100.0/sum(count(*)) over(),2) as percentage
from transcation_date_fact
group by 1

Payment_Types,percentage
WALLET,13.39
CASH ON DELIVERY,19.56
CREDIT CARD,19.87
DEBIT CARD,20.24
UPI,13.58
NET BANKING,13.36
